In [10]:
pip install datasets transformers torch evaluate nltk rouge_score

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import pandas as pd

In [13]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import datasets

In [14]:
csv_file_path = '/kaggle/input/majorfinal/MajorFinal.csv'

In [16]:
data=pd.read_csv(csv_file_path)

In [17]:
from datasets import Dataset

In [18]:
hf_dataset = Dataset.from_pandas(data)

In [19]:
hf_dataset

Dataset({
    features: ['Topic', 'Question', 'Answer'],
    num_rows: 50928
})

In [20]:
dataset = hf_dataset.train_test_split(test_size=0.2)

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Topic', 'Question', 'Answer'],
        num_rows: 40742
    })
    test: Dataset({
        features: ['Topic', 'Question', 'Answer'],
        num_rows: 10186
    })
})

In [22]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [23]:
prefix="answer the question: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Question"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text=examples["Answer"], max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [24]:
tokenized_data=dataset.map(preprocess_function, batched=True)

  0%|          | 0/41 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [25]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['Topic', 'Question', 'Answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 40742
    })
    test: Dataset({
        features: ['Topic', 'Question', 'Answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10186
    })
})

In [27]:
import torch

In [28]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Prepare data for BLEU and ROUGE
    decoded_preds = [pred.strip().split() for pred in decoded_preds]  # BLEU expects tokenized predictions
    decoded_labels = [[label.strip().split()] for label in decoded_labels]  # BLEU expects tokenized references in a specific format

    # Compute BLEU score
    bleu_score = corpus_bleu(decoded_labels, decoded_preds)

    # Compute ROUGE score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(' '.join(ref[0]), ' '.join(pred)) for ref, pred in zip(decoded_labels, decoded_preds)]
    rouge_scores = {key: np.mean([score[key].fmeasure for score in scores]) for key in ['rouge1', 'rouge2', 'rougeL']}

    # Combine metrics
    result = {**rouge_scores, "bleu_score": bleu_score}

    return result

In [29]:
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/tourist_festination/transformers/v3/1/T5-output-v3")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [30]:
named_params = list(model.named_parameters())
total_params = sum(p.numel() for p in model.parameters())


# Define the layers to freeze
layers_to_freeze = range(120)

#Freeze the specified layer
for index, (name, param) in enumerate(named_params):
    if index in layers_to_freeze:
        param.requires_grad = False

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# Print the freezing status and count of trainable parameters
for name, param in model.named_parameters():
    print(f"{name}, Requires Grad: {param.requires_grad}")

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

shared.weight, Requires Grad: False
encoder.block.0.layer.0.SelfAttention.q.weight, Requires Grad: False
encoder.block.0.layer.0.SelfAttention.k.weight, Requires Grad: False
encoder.block.0.layer.0.SelfAttention.v.weight, Requires Grad: False
encoder.block.0.layer.0.SelfAttention.o.weight, Requires Grad: False
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight, Requires Grad: False
encoder.block.0.layer.0.layer_norm.weight, Requires Grad: False
encoder.block.0.layer.1.DenseReluDense.wi.weight, Requires Grad: False
encoder.block.0.layer.1.DenseReluDense.wo.weight, Requires Grad: False
encoder.block.0.layer.1.layer_norm.weight, Requires Grad: False
encoder.block.1.layer.0.SelfAttention.q.weight, Requires Grad: False
encoder.block.1.layer.0.SelfAttention.k.weight, Requires Grad: False
encoder.block.1.layer.0.SelfAttention.v.weight, Requires Grad: False
encoder.block.1.layer.0.SelfAttention.o.weight, Requires Grad: False
encoder.block.1.layer.0.layer_norm.weight, Requires

In [31]:
model = model.to("cuda")

In [32]:
from transformers import EarlyStoppingCallback

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/T5-output-v3",
    logging_dir='/kaggle/working/T5-output-v3',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=25,
    fp16=False,
    predict_with_generate=True,
    report_to="tensorboard",
    load_best_model_at_end = True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_metrics=compute_metrics
)
trainer.train()
#trainer.train(resume_from_checkpoint=True)

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Bleu Score
1,3.149800,2.409011,0.329070,0.191199,0.291981,0.001776
2,3.113700,2.403725,0.328803,0.190596,0.291676,0.001824
3,3.122300,2.395063,0.328088,0.190513,0.290865,0.001724
4,3.107600,2.392349,0.329133,0.191082,0.291641,0.001792
5,3.089400,2.386342,0.330196,0.192045,0.292475,0.001829
6,3.068500,2.378666,0.331011,0.193209,0.293957,0.001841
7,3.067400,2.371014,0.330153,0.191795,0.292674,0.001803
8,3.071000,2.365820,0.329337,0.190761,0.292018,0.001812
9,3.048600,2.362070,0.329860,0.192380,0.293015,0.001765
10,3.040300,2.355956,0.329245,0.191346,0.292112,0.001825


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control

In [ ]:
trainer.save_model()

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [ ]:
zip_dir()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/T5-output-colab

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = '/content/T5-output'  # Model saved path

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small",use_fast=True)
model = T5ForConditionalGeneration.from_pretrained(model_path)


In [ ]:
question = "Suggest Top 15 places to visit in Nepal."  # Example question
input_text = "question: " + question  # Prefix the question

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids


In [ ]:
# Generate the answer
output_ids = model.generate(input_ids, max_length=512,top_p=0.9,top_k=0,do_sample=True,temperature=0.7,num_return_sequences=3,no_repeat_ngram_size=4)  # Adjust max_length

# Decode the generated ids to get the answer text
answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Answer:", answer)